# Multilingual Translated training

In this notebook we load previously translated train data, to several languages, train that data in a [multilingual bert model](https://huggingface.co/bert-base-multilingual-cased), and then predict on original portuguse data

Finally we gather all the translated train data into a big train set, 

In [1]:
!pip install pandas
!pip install datasets
!pip install transformers
!pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu113

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 346 kB 14.6 MB/s 
     |████████████████████████████████| 140 kB 72.8 MB/s 
     |████████████████████████████████| 212 kB 57.5 MB/s 
     |████████████████████████████████| 1.1 MB 67.7 MB/s 
     |████████████████████████████████| 86 kB 5.6 MB/s 
     |████████████████████████████████| 86 kB 7.4 MB/s 
     |████████████████████████████████| 596 kB 70.2 MB/s 
     |████████████████████████████████| 127 kB 56.2 MB/s 
     |████████████████████████████████| 94 kB 2.9 MB/s 
     |████████████████████████████████| 271 kB 53.8 MB/s 
     |████████████████████████████████| 144 kB 59.7 MB/s 
     |████████████████████████████████| 112 kB 52.2 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:

## Preparing our Data

### Loading dataset

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Load previously split dataset

In [3]:
import pandas as pd

def load_process_data(filename):
  dataset = pd.read_excel(filename)
  dataset = dataset.drop(columns=['article_id', 'annotator', 'node','ranges'])
  dataset.replace(['Value', 'Value(+)', 'Value(-)', 'Fact', 'Policy'],[0,1,2,3,4], inplace=True)

  return dataset

In [4]:
# 80% train
train_set = load_process_data('/content/drive/Shareddrives/PLN/Assignment 2/data/OpArticles_ADUs_train.xlsx')

# 20% test
test_set = load_process_data('/content/drive/Shareddrives/PLN/Assignment 2/data/OpArticles_ADUs_test.xlsx')

print(train_set.info())
print(test_set.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13394 entries, 0 to 13393
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   tokens  13394 non-null  object
 1   label   13394 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 209.4+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3349 entries, 0 to 3348
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   tokens  3349 non-null   object
 1   label   3349 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 52.5+ KB
None


Load translated train sets

In [5]:
train_en = load_process_data('/content/drive/Shareddrives/PLN/Assignment 2/data/multilingual/OpArticles_ADUs_train_en.xlsx')
train_es = load_process_data('/content/drive/Shareddrives/PLN/Assignment 2/data/multilingual/OpArticles_ADUs_train_es.xlsx')
train_ga = load_process_data('/content/drive/Shareddrives/PLN/Assignment 2/data/multilingual/OpArticles_ADUs_train_ga.xlsx')
train_fr = load_process_data('/content/drive/Shareddrives/PLN/Assignment 2/data/multilingual/OpArticles_ADUs_train_fr.xlsx')
train_it = load_process_data('/content/drive/Shareddrives/PLN/Assignment 2/data/multilingual/OpArticles_ADUs_train_it.xlsx')
train_ro = load_process_data('/content/drive/Shareddrives/PLN/Assignment 2/data/multilingual/OpArticles_ADUs_train_ro.xlsx')

train_en.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13394 entries, 0 to 13393
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   tokens  13394 non-null  object
 1   label   13394 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 209.4+ KB


For ease of usage with Transformer models, we convert the dataset into a Hugging Face dataset and split it into train, validation and test sets.

In [6]:
from datasets import Dataset
from datasets import DatasetDict

train_hf = Dataset.from_pandas(train_set)
test_hf = Dataset.from_pandas(test_set)

train_en_hf = Dataset.from_pandas(train_en)
train_es_hf = Dataset.from_pandas(train_es)
train_ga_hf = Dataset.from_pandas(train_ga)
train_fr_hf = Dataset.from_pandas(train_fr)
train_it_hf = Dataset.from_pandas(train_it)
train_ro_hf = Dataset.from_pandas(train_ro)

# Split 10% valid / 10% test
valid_test = test_hf.train_test_split(test_size=0.5, shuffle=True, seed=42)

train_valid_test_dataset = DatasetDict({
    'train_pt': train_hf,
    'validation': valid_test['train'],
    'test': valid_test['test'],
    'train_en': train_en_hf,
    'train_es': train_es_hf,
    'train_ga': train_ga_hf,
    'train_fr': train_fr_hf,
    'train_it': train_it_hf,
    'train_ro': train_ro_hf
})

train_valid_test_dataset

DatasetDict({
    train_pt: Dataset({
        features: ['tokens', 'label'],
        num_rows: 13394
    })
    validation: Dataset({
        features: ['tokens', 'label'],
        num_rows: 1674
    })
    test: Dataset({
        features: ['tokens', 'label'],
        num_rows: 1675
    })
    train_en: Dataset({
        features: ['tokens', 'label'],
        num_rows: 13394
    })
    train_es: Dataset({
        features: ['tokens', 'label'],
        num_rows: 13394
    })
    train_ga: Dataset({
        features: ['tokens', 'label'],
        num_rows: 13394
    })
    train_fr: Dataset({
        features: ['tokens', 'label'],
        num_rows: 13394
    })
    train_it: Dataset({
        features: ['tokens', 'label'],
        num_rows: 13394
    })
    train_ro: Dataset({
        features: ['tokens', 'label'],
        num_rows: 13394
    })
})

## Fine-tuning a pretrained model

### Tokenizer

We first load the tokenizer for our model:

In [7]:
from transformers import AutoTokenizer

def get_tokenizer(name):
    return AutoTokenizer.from_pretrained(name)

Now we need to [preprocess](https://huggingface.co/docs/transformers/preprocessing) our data.

Obtaining the length of the longest sequences in our data splits

In [8]:
def find_max_length(dataset):
    return len(max(dataset, key=lambda x: len(x.split())).split())

find_max_length(train_en_hf["tokens"])

86

Tokenize entire dataset

In [9]:
# Define tokenizer later
tokenizer = None

def tokenize_dataset(sample):
    return tokenizer(sample["tokens"], truncation=True, max_length=86, padding="max_length")

def get_tokenized_data(dataset):
    return dataset.map(tokenize_dataset, batched=True, remove_columns=["tokens"])

### Loading the model

Since we want to use the model for classification, we should load it with an appropriate classification head:

In [10]:
from transformers import AutoModelForSequenceClassification

model_name = "bert-base-multilingual-cased"

def get_model():
    model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=5)
    model.cuda() # Use GPU

    return model

### Fine-tuning

The next step is to [fine-tune](https://huggingface.co/docs/transformers/training) the model with our train data. To do so, we can make use of a [Trainer](https://huggingface.co/docs/transformers/main_classes/trainer).
There are several aspects of training that you can specify via [TrainingArguments](https://huggingface.co/docs/transformers/main_classes/trainer#transformers.TrainingArguments).

In [11]:
from transformers import TrainingArguments, Trainer
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
import numpy as np
from transformers import DataCollatorWithPadding
from torch import nn, tensor

class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        #inputs = inputs.to(device)
        labels = inputs.get("labels")
        # forward pass
        outputs = model(**inputs)
        logits = outputs.get("logits")
        # compute custom loss (5 labels with different weight)
        loss_fct = nn.CrossEntropyLoss(weight=tensor([0.41, 2.37, 1.15, 0.92, 5.01]))
        loss_fct.cuda()
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        return (loss, outputs) if return_outputs else loss

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='macro')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

def get_trainingArgs():
    return TrainingArguments(
        output_dir="./results",
        per_device_train_batch_size=32,
        per_device_eval_batch_size=32,
        fp16=True,
        learning_rate=2e-5,
        num_train_epochs=5,
        weight_decay=0.01,
        seed=42,
        data_seed=42,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
        metric_for_best_model="f1"
    )

def get_trainer(model_init_, args_, train_dataset_, eval_dataset_, tokenizer_, data_collator_, compute_metrics_):
    return CustomTrainer(
        model_init=model_init_,
        args=args_,
        train_dataset=train_dataset_,
        eval_dataset=eval_dataset_,
        tokenizer=tokenizer_,
        data_collator=data_collator_,
        compute_metrics=compute_metrics_
    )

### Train, evaluate, predict, and save

In [12]:
from transformers import DataCollatorWithPadding
from IPython.display import display

tokenizer = get_tokenizer(model_name)
tokenized_dataset = get_tokenized_data(train_valid_test_dataset)

def train_model(train_dataset_, lang='all'):
  trainer = get_trainer(
    get_model,
    get_trainingArgs(),
    train_dataset_,
    tokenized_dataset['validation'],
    tokenizer,
    DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics
    )
  
  # Train Model
  display(trainer.train())

  # Check performance in validation set
  display(trainer.evaluate())

  # Check how the model fares in our test set.
  display(trainer.predict(test_dataset=tokenized_dataset["test"]))

  # Save model for future use
  trainer.save_model('/content/drive/Shareddrives/PLN/Assignment 2/models/multilingual/' + lang + '/' + model_name)

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/972k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.87M [00:00<?, ?B/s]

  0%|          | 0/14 [00:00<?, ?ba/s]

...

  0%|          | 0/14 [00:00<?, ?ba/s]

  0%|          | 0/14 [00:00<?, ?ba/s]

  0%|          | 0/14 [00:00<?, ?ba/s]

  0%|          | 0/14 [00:00<?, ?ba/s]

  0%|          | 0/14 [00:00<?, ?ba/s]

### "Zero-shot" several languages

#### Portuguese

In [ ]:
train_model(train_dataset_=tokenized_dataset['train_pt'], lang='pt')

loading configuration file https://huggingface.co/bert-base-multilingual-cased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/6c4a5d81a58c9791cdf76a09bce1b5abfb9cf958aebada51200f4515403e5d08.0fe59f3f4f1335dadeb4bce8b8146199d9083512b50d07323c1c319f96df450c
Model config BertConfig {
  "_name_or_path": "bert-base-multilingual-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,No log,1.055092,0.456392,0.470952,0.455387,0.601867
2,1.270800,1.042111,0.510155,0.526929,0.513309,0.622323
3,0.944300,1.038146,0.499403,0.517390,0.485653,0.628687
4,0.742200,1.103155,0.504779,0.527785,0.499376,0.616708
5,0.609200,1.153276,0.520908,0.538613,0.505483,0.609511


***** Running Evaluation *****
  Num examples = 1674
  Batch size = 32
Saving model checkpoint to ./results/checkpoint-419
Configuration saved in ./results/checkpoint-419/config.json
Model weights saved in ./results/checkpoint-419/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-419/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-419/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 1674
  Batch size = 32
Saving model checkpoint to ./results/checkpoint-838
Configuration saved in ./results/checkpoint-838/config.json
Model weights saved in ./results/checkpoint-838/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-838/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-838/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 1674
  Batch size = 32
Saving model checkpoint to ./results/checkpoint-1257
Configuration saved in ./results/checkpoint-1257/config.json
Model

TrainOutput(global_step=2095, training_loss=0.8773514212742626, metrics={'train_runtime': 514.8779, 'train_samples_per_second': 130.07, 'train_steps_per_second': 4.069, 'total_flos': 2959781039199240.0, 'train_loss': 0.8773514212742626, 'epoch': 5.0})

***** Running Evaluation *****
  Num examples = 1674
  Batch size = 32


{'epoch': 5.0,
 'eval_accuracy': 0.5209080047789725,
 'eval_f1': 0.5386127302795233,
 'eval_loss': 1.1532762050628662,
 'eval_precision': 0.5054825144919052,
 'eval_recall': 0.609510904741909,
 'eval_runtime': 2.9511,
 'eval_samples_per_second': 567.238,
 'eval_steps_per_second': 17.959}

***** Running Prediction *****
  Num examples = 1675
  Batch size = 32


PredictionOutput(predictions=array([[ 1.782  ,  0.0139 , -1.514  ,  2.771  , -3.049  ],
       [ 2.08   , -0.705  ,  1.144  ,  0.674  , -2.645  ],
       [ 0.732  , -2.635  ,  3.81   ,  0.2216 , -1.407  ],
       ...,
       [ 0.765  ,  3.477  , -2.652  ,  0.951  , -2.059  ],
       [ 0.6016 , -2.21   ,  3.982  ,  0.03102, -1.922  ],
       [ 0.067  , -0.9795 , -2.082  , -1.959  ,  4.434  ]], dtype=float16), label_ids=array([0, 3, 2, ..., 1, 2, 4]), metrics={'test_loss': 1.2595195770263672, 'test_accuracy': 0.5355223880597015, 'test_f1': 0.5295883701713505, 'test_precision': 0.5078855183163504, 'test_recall': 0.5760040009130918, 'test_runtime': 2.9917, 'test_samples_per_second': 559.878, 'test_steps_per_second': 17.716})

Saving model checkpoint to /content/drive/Shareddrives/PLN/Assignment 2/models/multilingual/pt/bert-base-multilingual-cased
Configuration saved in /content/drive/Shareddrives/PLN/Assignment 2/models/multilingual/pt/bert-base-multilingual-cased/config.json
Model weights saved in /content/drive/Shareddrives/PLN/Assignment 2/models/multilingual/pt/bert-base-multilingual-cased/pytorch_model.bin
tokenizer config file saved in /content/drive/Shareddrives/PLN/Assignment 2/models/multilingual/pt/bert-base-multilingual-cased/tokenizer_config.json
Special tokens file saved in /content/drive/Shareddrives/PLN/Assignment 2/models/multilingual/pt/bert-base-multilingual-cased/special_tokens_map.json


In [ ]:
!rm -rf ./results/

#### English

In [ ]:
train_model(train_dataset_=tokenized_dataset['train_en'], lang='en')

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
loading configuration file https://huggingface.co/bert-base-multilingual-cased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/6c4a5d81a58c9791cdf76a09bce1b5abfb9cf958aebada51200f4515403e5d08.0fe59f3f4f1335dadeb4bce8b8146199d9083512b50d07323c1c319f96df450c
Model config BertConfig {
  "_name_or_path": "bert-base-multilingual-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4"
  },


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,No log,1.147104,0.465352,0.476734,0.469287,0.559196
2,1.268200,1.202941,0.446834,0.470704,0.495770,0.539120
3,0.922400,1.194357,0.482676,0.500826,0.485800,0.564963
4,0.719900,1.295992,0.485663,0.492740,0.484505,0.539632
5,0.605900,1.384297,0.500597,0.481143,0.491887,0.502393


***** Running Evaluation *****
  Num examples = 1674
  Batch size = 32
Saving model checkpoint to ./results/checkpoint-419
Configuration saved in ./results/checkpoint-419/config.json
Model weights saved in ./results/checkpoint-419/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-419/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-419/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 1674
  Batch size = 32
Saving model checkpoint to ./results/checkpoint-838
Configuration saved in ./results/checkpoint-838/config.json
Model weights saved in ./results/checkpoint-838/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-838/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-838/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 1674
  Batch size = 32
Saving model checkpoint to ./results/checkpoint-1257
Configuration saved in ./results/checkpoint-1257/config.json
Model

TrainOutput(global_step=2095, training_loss=0.8649118464431217, metrics={'train_runtime': 518.0657, 'train_samples_per_second': 129.269, 'train_steps_per_second': 4.044, 'total_flos': 2959781039199240.0, 'train_loss': 0.8649118464431217, 'epoch': 5.0})

***** Running Evaluation *****
  Num examples = 1674
  Batch size = 32


{'epoch': 5.0,
 'eval_accuracy': 0.4826762246117085,
 'eval_f1': 0.5008261525649764,
 'eval_loss': 1.194356918334961,
 'eval_precision': 0.48580042415958424,
 'eval_recall': 0.5649630822294702,
 'eval_runtime': 2.9669,
 'eval_samples_per_second': 564.233,
 'eval_steps_per_second': 17.864}

***** Running Prediction *****
  Num examples = 1675
  Batch size = 32


PredictionOutput(predictions=array([[ 1.752 ,  0.7065, -0.581 ,  1.742 , -2.936 ],
       [ 2.014 , -1.0205,  1.605 ,  1.304 , -3.242 ],
       [ 1.037 , -2.15  ,  3.18  ,  0.1595, -1.714 ],
       ...,
       [ 0.69  ,  3.254 , -1.968 ,  0.4653, -2.162 ],
       [ 0.6597,  0.1566, -0.703 ,  1.992 , -1.791 ],
       [ 0.9995,  2.027 , -2.053 , -1.902 ,  1.239 ]], dtype=float16), label_ids=array([0, 3, 2, ..., 1, 2, 4]), metrics={'test_loss': 1.2670106887817383, 'test_accuracy': 0.4943283582089552, 'test_f1': 0.48871851888196705, 'test_precision': 0.47777741687277375, 'test_recall': 0.5412165213983396, 'test_runtime': 2.9852, 'test_samples_per_second': 561.094, 'test_steps_per_second': 17.754})

Saving model checkpoint to /content/drive/Shareddrives/PLN/Assignment 2/models/multilingual/en/bert-base-multilingual-cased
Configuration saved in /content/drive/Shareddrives/PLN/Assignment 2/models/multilingual/en/bert-base-multilingual-cased/config.json
Model weights saved in /content/drive/Shareddrives/PLN/Assignment 2/models/multilingual/en/bert-base-multilingual-cased/pytorch_model.bin
tokenizer config file saved in /content/drive/Shareddrives/PLN/Assignment 2/models/multilingual/en/bert-base-multilingual-cased/tokenizer_config.json
Special tokens file saved in /content/drive/Shareddrives/PLN/Assignment 2/models/multilingual/en/bert-base-multilingual-cased/special_tokens_map.json


In [ ]:
!rm -rf ./results/

#### Spanish

In [ ]:
train_model(train_dataset_=tokenized_dataset['train_es'], lang='es')

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
loading configuration file https://huggingface.co/bert-base-multilingual-cased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/6c4a5d81a58c9791cdf76a09bce1b5abfb9cf958aebada51200f4515403e5d08.0fe59f3f4f1335dadeb4bce8b8146199d9083512b50d07323c1c319f96df450c
Model config BertConfig {
  "_name_or_path": "bert-base-multilingual-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4"
  },


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,No log,1.125831,0.476105,0.467229,0.449610,0.535514
2,1.247200,1.160231,0.473118,0.493518,0.510417,0.567072
3,0.919400,1.105890,0.494624,0.499510,0.471640,0.592932


***** Running Evaluation *****
  Num examples = 1674
  Batch size = 32
Saving model checkpoint to ./results/checkpoint-419
Configuration saved in ./results/checkpoint-419/config.json
Model weights saved in ./results/checkpoint-419/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-419/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-419/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 1674
  Batch size = 32
Saving model checkpoint to ./results/checkpoint-838
Configuration saved in ./results/checkpoint-838/config.json
Model weights saved in ./results/checkpoint-838/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-838/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-838/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 1674
  Batch size = 32
Saving model checkpoint to ./results/checkpoint-1257
Configuration saved in ./results/checkpoint-1257/config.json
Model

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,No log,1.125831,0.476105,0.467229,0.449610,0.535514
2,1.247200,1.160231,0.473118,0.493518,0.510417,0.567072
3,0.919400,1.105890,0.494624,0.499510,0.471640,0.592932
4,0.722600,1.219127,0.488053,0.503328,0.484944,0.574008
5,0.588600,1.249187,0.516129,0.513435,0.492072,0.561575


***** Running Evaluation *****
  Num examples = 1674
  Batch size = 32
Saving model checkpoint to ./results/checkpoint-1676
Configuration saved in ./results/checkpoint-1676/config.json
Model weights saved in ./results/checkpoint-1676/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-1676/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-1676/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 1674
  Batch size = 32
Saving model checkpoint to ./results/checkpoint-2095
Configuration saved in ./results/checkpoint-2095/config.json
Model weights saved in ./results/checkpoint-2095/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-2095/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-2095/special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from ./results/checkpoint-2095 (score: 0.5134354399816512).


...

***** Running Evaluation *****
  Num examples = 1674
  Batch size = 32


{'epoch': 5.0,
 'eval_accuracy': 0.5161290322580645,
 'eval_f1': 0.5134354399816512,
 'eval_loss': 1.2491865158081055,
 'eval_precision': 0.4920719323602058,
 'eval_recall': 0.5615752917069609,
 'eval_runtime': 2.9438,
 'eval_samples_per_second': 568.647,
 'eval_steps_per_second': 18.004}

***** Running Prediction *****
  Num examples = 1675
  Batch size = 32


PredictionOutput(predictions=array([[ 1.688 , -1.494 , -1.297 ,  3.178 , -2.607 ],
       [ 2.117 , -0.322 ,  2.023 , -0.2686, -2.639 ],
       [ 1.41  , -1.952 ,  3.133 ,  0.103 , -2.125 ],
       ...,
       [ 0.801 ,  3.902 , -2.23  ,  0.516 , -2.375 ],
       [ 1.091 , -1.404 ,  1.092 ,  1.1   , -1.94  ],
       [ 0.8633,  2.291 , -2.72  , -2.22  ,  2.045 ]], dtype=float16), label_ids=array([0, 3, 2, ..., 1, 2, 4]), metrics={'test_loss': 1.36050546169281, 'test_accuracy': 0.5164179104477612, 'test_f1': 0.5010973562651306, 'test_precision': 0.48625386627340417, 'test_recall': 0.5389673708764617, 'test_runtime': 3.0177, 'test_samples_per_second': 555.066, 'test_steps_per_second': 17.563})

Saving model checkpoint to /content/drive/Shareddrives/PLN/Assignment 2/models/multilingual/es/bert-base-multilingual-cased
Configuration saved in /content/drive/Shareddrives/PLN/Assignment 2/models/multilingual/es/bert-base-multilingual-cased/config.json
Model weights saved in /content/drive/Shareddrives/PLN/Assignment 2/models/multilingual/es/bert-base-multilingual-cased/pytorch_model.bin
tokenizer config file saved in /content/drive/Shareddrives/PLN/Assignment 2/models/multilingual/es/bert-base-multilingual-cased/tokenizer_config.json
Special tokens file saved in /content/drive/Shareddrives/PLN/Assignment 2/models/multilingual/es/bert-base-multilingual-cased/special_tokens_map.json


In [ ]:
!rm -rf ./results/

#### Galician

In [ ]:
train_model(train_dataset_=tokenized_dataset['train_ga'], lang='ga')

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
loading configuration file https://huggingface.co/bert-base-multilingual-cased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/6c4a5d81a58c9791cdf76a09bce1b5abfb9cf958aebada51200f4515403e5d08.0fe59f3f4f1335dadeb4bce8b8146199d9083512b50d07323c1c319f96df450c
Model config BertConfig {
  "_name_or_path": "bert-base-multilingual-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4"
  },


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,No log,1.112419,0.437873,0.454308,0.454785,0.553492
2,1.312300,1.125961,0.455197,0.467657,0.479933,0.565619
3,0.970000,1.127531,0.508961,0.518939,0.498897,0.587071
4,0.773900,1.205605,0.489247,0.493722,0.470250,0.562820
5,0.637300,1.245800,0.520311,0.518363,0.496968,0.565666


***** Running Evaluation *****
  Num examples = 1674
  Batch size = 32
Saving model checkpoint to ./results/checkpoint-419
Configuration saved in ./results/checkpoint-419/config.json
Model weights saved in ./results/checkpoint-419/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-419/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-419/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 1674
  Batch size = 32
Saving model checkpoint to ./results/checkpoint-838
Configuration saved in ./results/checkpoint-838/config.json
Model weights saved in ./results/checkpoint-838/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-838/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-838/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 1674
  Batch size = 32
Saving model checkpoint to ./results/checkpoint-1257
Configuration saved in ./results/checkpoint-1257/config.json
Model

TrainOutput(global_step=2095, training_loss=0.9086488637264133, metrics={'train_runtime': 519.1956, 'train_samples_per_second': 128.988, 'train_steps_per_second': 4.035, 'total_flos': 2959781039199240.0, 'train_loss': 0.9086488637264133, 'epoch': 5.0})

***** Running Evaluation *****
  Num examples = 1674
  Batch size = 32


{'epoch': 5.0,
 'eval_accuracy': 0.5089605734767025,
 'eval_f1': 0.5189385845699575,
 'eval_loss': 1.1275306940078735,
 'eval_precision': 0.49889713400551194,
 'eval_recall': 0.5870714759933998,
 'eval_runtime': 2.9397,
 'eval_samples_per_second': 569.437,
 'eval_steps_per_second': 18.029}

***** Running Prediction *****
  Num examples = 1675
  Batch size = 32


PredictionOutput(predictions=array([[ 1.391 ,  0.4922, -0.6616,  2.281 , -3.662 ],
       [ 1.457 , -0.2883,  1.469 ,  1.312 , -3.72  ],
       [ 0.6313, -1.884 ,  3.066 ,  0.1538, -1.694 ],
       ...,
       [ 0.945 ,  3.1   , -1.585 ,  0.5664, -2.277 ],
       [ 0.3367, -1.425 ,  1.989 ,  0.1013, -1.433 ],
       [ 1.243 ,  0.1583, -1.746 , -1.929 ,  1.907 ]], dtype=float16), label_ids=array([0, 3, 2, ..., 1, 2, 4]), metrics={'test_loss': 1.2426650524139404, 'test_accuracy': 0.4943283582089552, 'test_f1': 0.4941026913955997, 'test_precision': 0.4791878227182047, 'test_recall': 0.5555881914972824, 'test_runtime': 3.0065, 'test_samples_per_second': 557.129, 'test_steps_per_second': 17.629})

Saving model checkpoint to /content/drive/Shareddrives/PLN/Assignment 2/models/multilingual/ga/bert-base-multilingual-cased
Configuration saved in /content/drive/Shareddrives/PLN/Assignment 2/models/multilingual/ga/bert-base-multilingual-cased/config.json
Model weights saved in /content/drive/Shareddrives/PLN/Assignment 2/models/multilingual/ga/bert-base-multilingual-cased/pytorch_model.bin
tokenizer config file saved in /content/drive/Shareddrives/PLN/Assignment 2/models/multilingual/ga/bert-base-multilingual-cased/tokenizer_config.json
Special tokens file saved in /content/drive/Shareddrives/PLN/Assignment 2/models/multilingual/ga/bert-base-multilingual-cased/special_tokens_map.json


In [ ]:
!rm -rf ./results/

#### French

In [ ]:
train_model(train_dataset_=tokenized_dataset['train_fr'], lang='fr')

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
loading configuration file https://huggingface.co/bert-base-multilingual-cased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/6c4a5d81a58c9791cdf76a09bce1b5abfb9cf958aebada51200f4515403e5d08.0fe59f3f4f1335dadeb4bce8b8146199d9083512b50d07323c1c319f96df450c
Model config BertConfig {
  "_name_or_path": "bert-base-multilingual-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4"
  },


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,No log,1.157611,0.456392,0.450605,0.423860,0.550167
2,1.270000,1.179483,0.459379,0.448009,0.441550,0.551234
3,0.925300,1.193189,0.469534,0.480394,0.463030,0.556859


***** Running Evaluation *****
  Num examples = 1674
  Batch size = 32
Saving model checkpoint to ./results/checkpoint-419
Configuration saved in ./results/checkpoint-419/config.json
Model weights saved in ./results/checkpoint-419/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-419/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-419/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 1674
  Batch size = 32
Saving model checkpoint to ./results/checkpoint-838
Configuration saved in ./results/checkpoint-838/config.json
Model weights saved in ./results/checkpoint-838/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-838/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-838/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 1674
  Batch size = 32
Saving model checkpoint to ./results/checkpoint-1257
Configuration saved in ./results/checkpoint-1257/config.json
Model

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,No log,1.157611,0.456392,0.450605,0.423860,0.550167
2,1.270000,1.179483,0.459379,0.448009,0.441550,0.551234
3,0.925300,1.193189,0.469534,0.480394,0.463030,0.556859
4,0.720300,1.273748,0.465352,0.464721,0.439850,0.529035
5,0.611800,1.324338,0.493429,0.480979,0.460788,0.524503


***** Running Evaluation *****
  Num examples = 1674
  Batch size = 32
Saving model checkpoint to ./results/checkpoint-1676
Configuration saved in ./results/checkpoint-1676/config.json
Model weights saved in ./results/checkpoint-1676/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-1676/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-1676/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 1674
  Batch size = 32
Saving model checkpoint to ./results/checkpoint-2095
Configuration saved in ./results/checkpoint-2095/config.json
Model weights saved in ./results/checkpoint-2095/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-2095/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-2095/special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from ./results/checkpoint-2095 (score: 0.48097852967740895).


...

***** Running Evaluation *****
  Num examples = 1674
  Batch size = 32


{'epoch': 5.0,
 'eval_accuracy': 0.4934289127837515,
 'eval_f1': 0.48097852967740895,
 'eval_loss': 1.3243380784988403,
 'eval_precision': 0.46078764036114056,
 'eval_recall': 0.5245033712031266,
 'eval_runtime': 2.9586,
 'eval_samples_per_second': 565.804,
 'eval_steps_per_second': 17.914}

***** Running Prediction *****
  Num examples = 1675
  Batch size = 32


PredictionOutput(predictions=array([[ 1.963  ,  0.08466, -1.167  ,  1.954  , -2.893  ],
       [ 1.447  , -1.728  ,  1.789  ,  1.818  , -3.197  ],
       [ 0.446  , -2.545  ,  3.348  ,  0.1467 , -1.432  ],
       ...,
       [ 0.7515 ,  3.633  , -2.621  ,  0.4343 , -1.848  ],
       [ 0.92   , -2.742  ,  2.568  ,  0.816  , -1.716  ],
       [ 1.012  , -0.6562 , -2.055  , -2.006  ,  3.645  ]], dtype=float16), label_ids=array([0, 3, 2, ..., 1, 2, 4]), metrics={'test_loss': 1.293208122253418, 'test_accuracy': 0.5080597014925373, 'test_f1': 0.4949213385620916, 'test_precision': 0.4726470595961206, 'test_recall': 0.5530535966899603, 'test_runtime': 3.0138, 'test_samples_per_second': 555.769, 'test_steps_per_second': 17.586})

Saving model checkpoint to /content/drive/Shareddrives/PLN/Assignment 2/models/multilingual/fr/bert-base-multilingual-cased
Configuration saved in /content/drive/Shareddrives/PLN/Assignment 2/models/multilingual/fr/bert-base-multilingual-cased/config.json
Model weights saved in /content/drive/Shareddrives/PLN/Assignment 2/models/multilingual/fr/bert-base-multilingual-cased/pytorch_model.bin
tokenizer config file saved in /content/drive/Shareddrives/PLN/Assignment 2/models/multilingual/fr/bert-base-multilingual-cased/tokenizer_config.json
Special tokens file saved in /content/drive/Shareddrives/PLN/Assignment 2/models/multilingual/fr/bert-base-multilingual-cased/special_tokens_map.json


In [ ]:
!rm -rf ./results/

#### Italian

In [ ]:
train_model(train_dataset_=tokenized_dataset['train_it'], lang='it')

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
loading configuration file https://huggingface.co/bert-base-multilingual-cased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/6c4a5d81a58c9791cdf76a09bce1b5abfb9cf958aebada51200f4515403e5d08.0fe59f3f4f1335dadeb4bce8b8146199d9083512b50d07323c1c319f96df450c
Model config BertConfig {
  "_name_or_path": "bert-base-multilingual-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4"
  },


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,No log,1.123776,0.467742,0.465785,0.442078,0.566229
2,1.266200,1.136467,0.459976,0.469226,0.475408,0.565809
3,0.941400,1.138143,0.461768,0.473976,0.459842,0.566262
4,0.757000,1.205573,0.492832,0.501338,0.479627,0.560441
5,0.612300,1.273915,0.513142,0.514780,0.498503,0.548238


***** Running Evaluation *****
  Num examples = 1674
  Batch size = 32
Saving model checkpoint to ./results/checkpoint-419
Configuration saved in ./results/checkpoint-419/config.json
Model weights saved in ./results/checkpoint-419/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-419/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-419/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 1674
  Batch size = 32
Saving model checkpoint to ./results/checkpoint-838
Configuration saved in ./results/checkpoint-838/config.json
Model weights saved in ./results/checkpoint-838/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-838/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-838/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 1674
  Batch size = 32
Saving model checkpoint to ./results/checkpoint-1257
Configuration saved in ./results/checkpoint-1257/config.json
Model

TrainOutput(global_step=2095, training_loss=0.8802229997366311, metrics={'train_runtime': 515.8011, 'train_samples_per_second': 129.837, 'train_steps_per_second': 4.062, 'total_flos': 2959781039199240.0, 'train_loss': 0.8802229997366311, 'epoch': 5.0})

***** Running Evaluation *****
  Num examples = 1674
  Batch size = 32


{'epoch': 5.0,
 'eval_accuracy': 0.513142174432497,
 'eval_f1': 0.5147802604716135,
 'eval_loss': 1.2739146947860718,
 'eval_precision': 0.498502572205277,
 'eval_recall': 0.548238324707287,
 'eval_runtime': 2.9571,
 'eval_samples_per_second': 566.089,
 'eval_steps_per_second': 17.923}

***** Running Prediction *****
  Num examples = 1675
  Batch size = 32


PredictionOutput(predictions=array([[ 1.501 ,  0.7534, -2.31  ,  2.299 , -2.705 ],
       [ 1.441 , -0.906 ,  0.2954,  2.354 , -3.463 ],
       [ 0.6025, -2.287 ,  3.592 ,  0.4968, -1.693 ],
       ...,
       [ 1.22  ,  3.143 , -2.512 ,  0.857 , -2.402 ],
       [ 0.9834, -0.0684,  2.082 , -0.7183, -2.07  ],
       [ 1.226 , -0.5513, -1.811 , -2.428 ,  3.545 ]], dtype=float16), label_ids=array([0, 3, 2, ..., 1, 2, 4]), metrics={'test_loss': 1.3433645963668823, 'test_accuracy': 0.5164179104477612, 'test_f1': 0.4942627616753216, 'test_precision': 0.4774481064499514, 'test_recall': 0.5345071251434887, 'test_runtime': 2.9932, 'test_samples_per_second': 559.607, 'test_steps_per_second': 17.707})

Saving model checkpoint to /content/drive/Shareddrives/PLN/Assignment 2/models/multilingual/it/bert-base-multilingual-cased
Configuration saved in /content/drive/Shareddrives/PLN/Assignment 2/models/multilingual/it/bert-base-multilingual-cased/config.json
Model weights saved in /content/drive/Shareddrives/PLN/Assignment 2/models/multilingual/it/bert-base-multilingual-cased/pytorch_model.bin
tokenizer config file saved in /content/drive/Shareddrives/PLN/Assignment 2/models/multilingual/it/bert-base-multilingual-cased/tokenizer_config.json
Special tokens file saved in /content/drive/Shareddrives/PLN/Assignment 2/models/multilingual/it/bert-base-multilingual-cased/special_tokens_map.json


In [ ]:
!rm -rf ./results/

#### Romanian

In [13]:
train_model(train_dataset_=tokenized_dataset['train_ro'], lang='ro')

loading configuration file https://huggingface.co/bert-base-multilingual-cased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/6c4a5d81a58c9791cdf76a09bce1b5abfb9cf958aebada51200f4515403e5d08.0fe59f3f4f1335dadeb4bce8b8146199d9083512b50d07323c1c319f96df450c
Model config BertConfig {
  "_name_or_path": "bert-base-multilingual-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_

Downloading:   0%|          | 0.00/681M [00:00<?, ?B/s]

storing https://huggingface.co/bert-base-multilingual-cased/resolve/main/pytorch_model.bin in cache at /root/.cache/huggingface/transformers/0a3fd51713dcbb4def175c7f85bddc995d5976ce1dde327f99104e4d33069f17.aa7be4c79d76f4066d9b354496ea477c9ee39c5d889156dd1efb680643c2b052
creating metadata file for /root/.cache/huggingface/transformers/0a3fd51713dcbb4def175c7f85bddc995d5976ce1dde327f99104e4d33069f17.aa7be4c79d76f4066d9b354496ea477c9ee39c5d889156dd1efb680643c2b052
loading weights file https://huggingface.co/bert-base-multilingual-cased/resolve/main/pytorch_model.bin from cache at /root/.cache/huggingface/transformers/0a3fd51713dcbb4def175c7f85bddc995d5976ce1dde327f99104e4d33069f17.aa7be4c79d76f4066d9b354496ea477c9ee39c5d889156dd1efb680643c2b052
Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.s

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,No log,1.205533,0.405615,0.413493,0.416707,0.514102
2,1.303100,1.227110,0.454600,0.435044,0.433085,0.518966
3,0.988800,1.265098,0.430108,0.429052,0.420486,0.502340
4,0.792200,1.418741,0.439068,0.418005,0.417162,0.476670
5,0.658200,1.482365,0.475508,0.432354,0.430032,0.469629


***** Running Evaluation *****
  Num examples = 1674
  Batch size = 32
Saving model checkpoint to ./results/checkpoint-419
Configuration saved in ./results/checkpoint-419/config.json
Model weights saved in ./results/checkpoint-419/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-419/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-419/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 1674
  Batch size = 32
Saving model checkpoint to ./results/checkpoint-838
Configuration saved in ./results/checkpoint-838/config.json
Model weights saved in ./results/checkpoint-838/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-838/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-838/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 1674
  Batch size = 32
Saving model checkpoint to ./results/checkpoint-1257
Configuration saved in ./results/checkpoint-1257/config.json
Model

...

***** Running Evaluation *****
  Num examples = 1674
  Batch size = 32


{'epoch': 5.0,
 'eval_accuracy': 0.45459976105137395,
 'eval_f1': 0.43504380422438527,
 'eval_loss': 1.2271100282669067,
 'eval_precision': 0.4330854722631917,
 'eval_recall': 0.5189658131496376,
 'eval_runtime': 2.9149,
 'eval_samples_per_second': 574.285,
 'eval_steps_per_second': 18.182}

***** Running Prediction *****
  Num examples = 1675
  Batch size = 32


PredictionOutput(predictions=array([[ 1.386 ,  0.369 , -0.1338,  1.121 , -1.93  ],
       [ 0.9824, -0.631 ,  1.694 ,  0.444 , -2.242 ],
       [ 0.5024, -1.35  ,  2.238 , -0.1741, -0.7397],
       ...,
       [ 0.847 ,  2.111 , -0.965 ,  0.613 , -1.581 ],
       [ 0.1373, -0.4993,  0.561 ,  0.0692, -0.506 ],
       [ 0.3225, -0.2546, -1.834 , -1.601 ,  3.285 ]], dtype=float16), label_ids=array([0, 3, 2, ..., 1, 2, 4]), metrics={'test_loss': 1.208327054977417, 'test_accuracy': 0.47044776119402987, 'test_f1': 0.45284103360058686, 'test_precision': 0.4502556794813013, 'test_recall': 0.5304726240180785, 'test_runtime': 2.9642, 'test_samples_per_second': 565.084, 'test_steps_per_second': 17.88})

Saving model checkpoint to /content/drive/Shareddrives/PLN/Assignment 2/models/multilingual/ro/bert-base-multilingual-cased
Configuration saved in /content/drive/Shareddrives/PLN/Assignment 2/models/multilingual/ro/bert-base-multilingual-cased/config.json
Model weights saved in /content/drive/Shareddrives/PLN/Assignment 2/models/multilingual/ro/bert-base-multilingual-cased/pytorch_model.bin
tokenizer config file saved in /content/drive/Shareddrives/PLN/Assignment 2/models/multilingual/ro/bert-base-multilingual-cased/tokenizer_config.json
Special tokens file saved in /content/drive/Shareddrives/PLN/Assignment 2/models/multilingual/ro/bert-base-multilingual-cased/special_tokens_map.json


In [14]:
!rm -rf ./results/

### Multilingual train dataset

In [ ]:
from datasets import concatenate_datasets

tokenized_train_all = concatenate_datasets([
                                            tokenized_dataset['train_pt'],
                                            tokenized_dataset['train_en'],
                                            tokenized_dataset['train_es'],
                                            tokenized_dataset['train_ga'],
                                            tokenized_dataset['train_fr'],
                                            tokenized_dataset['train_it'],
                                            tokenized_dataset['train_ro']
                                            ])

In [ ]:
tokenized_train_all

In [ ]:
train_model(train_dataset_=tokenized_train_all)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
loading configuration file https://huggingface.co/bert-base-multilingual-cased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/6c4a5d81a58c9791cdf76a09bce1b5abfb9cf958aebada51200f4515403e5d08.0fe59f3f4f1335dadeb4bce8b8146199d9083512b50d07323c1c319f96df450c
Model config BertConfig {
  "_name_or_path": "bert-base-multilingual-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4"
  },


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.882700,1.148077,0.474910,0.496210,0.474372,0.603743
2,0.601100,1.358612,0.514337,0.539555,0.513306,0.593673
3,0.419600,1.642012,0.538829,0.546187,0.525290,0.576941
4,0.306700,1.987910,0.552569,0.557276,0.542059,0.576649
5,0.249800,2.168152,0.551374,0.554634,0.535602,0.578899


***** Running Evaluation *****
  Num examples = 1674
  Batch size = 32
Saving model checkpoint to ./results/checkpoint-2930
Configuration saved in ./results/checkpoint-2930/config.json
Model weights saved in ./results/checkpoint-2930/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-2930/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-2930/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 1674
  Batch size = 32
Saving model checkpoint to ./results/checkpoint-5860
Configuration saved in ./results/checkpoint-5860/config.json
Model weights saved in ./results/checkpoint-5860/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-5860/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-5860/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 1674
  Batch size = 32
Saving model checkpoint to ./results/checkpoint-8790
Configuration saved in ./results/checkpoint-8790/config.

TrainOutput(global_step=14650, training_loss=0.5344101103173995, metrics={'train_runtime': 3268.6009, 'train_samples_per_second': 143.422, 'train_steps_per_second': 4.482, 'total_flos': 2.071846727439468e+16, 'train_loss': 0.5344101103173995, 'epoch': 5.0})

***** Running Evaluation *****
  Num examples = 1674
  Batch size = 32


{'epoch': 5.0,
 'eval_accuracy': 0.5525686977299881,
 'eval_f1': 0.5572762272658605,
 'eval_loss': 1.9879100322723389,
 'eval_precision': 0.5420593574146314,
 'eval_recall': 0.5766486596820652,
 'eval_runtime': 2.9177,
 'eval_samples_per_second': 573.744,
 'eval_steps_per_second': 18.165}

***** Running Prediction *****
  Num examples = 1675
  Batch size = 32


PredictionOutput(predictions=array([[ 3.871 , -2.443 , -1.864 ,  4.203 , -4.555 ],
       [ 2.87  , -2.117 ,  4.605 , -1.571 , -3.152 ],
       [ 1.193 , -4.17  ,  4.285 ,  2.365 , -3.482 ],
       ...,
       [ 1.748 ,  4.773 , -4.125 ,  0.687 , -3.201 ],
       [ 1.596 , -3.758 ,  5.516 ,  0.1616, -3.197 ],
       [ 0.7017, -1.011 , -2.432 , -3.457 ,  6.145 ]], dtype=float16), label_ids=array([0, 3, 2, ..., 1, 2, 4]), metrics={'test_loss': 2.11346435546875, 'test_accuracy': 0.5498507462686567, 'test_f1': 0.5361711217079529, 'test_precision': 0.527777279848123, 'test_recall': 0.5480669338851157, 'test_runtime': 2.9654, 'test_samples_per_second': 564.849, 'test_steps_per_second': 17.873})

Saving model checkpoint to /content/drive/Shareddrives/PLN/Assignment 2/models/multilingual/all/bert-base-multilingual-cased
Configuration saved in /content/drive/Shareddrives/PLN/Assignment 2/models/multilingual/all/bert-base-multilingual-cased/config.json
Model weights saved in /content/drive/Shareddrives/PLN/Assignment 2/models/multilingual/all/bert-base-multilingual-cased/pytorch_model.bin
tokenizer config file saved in /content/drive/Shareddrives/PLN/Assignment 2/models/multilingual/all/bert-base-multilingual-cased/tokenizer_config.json
Special tokens file saved in /content/drive/Shareddrives/PLN/Assignment 2/models/multilingual/all/bert-base-multilingual-cased/special_tokens_map.json


In [ ]:
!rm -rf ./results/